In [1]:
import numpy as np
import pandas as pd
import dataframe_image as dfi

#import pprint as pp
from datetime import datetime, timedelta

%load_ext autoreload
%autoreload 2

#to display offline interactive plots
import plotly as py

# mapping library
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
#from urllib.request import urlopen
#import json

# to export plots to html
#import plotly.io as pio

import plotly.io as pio
pio.renderers.default='notebook'

# hide annoying repeated deprec warnings (statsmodel issue)
import warnings
warnings.simplefilter('once', category=UserWarning)

# load my data processing modules
import process_test_data as pt
import process_hosp_data as hd
import process_kpi as kpi
from process_metro_bubble import to_chart_studio

# fix slowness issue? See https://github.com/microsoft/vscode-python/issues/10998
%config Completer.use_jedi = False

# Load data

In [2]:
## get base kpi df
kpi_df = pd.read_pickle('data/latest_kpi.pkl')

# add cols for overall & 70+ testing rate to kpi_df
test_df = pt.create_testing_df()
test_df = pt.create_rolling_cols(test_df)[['libelle_dep', 'jour', 'rolling_test_100k']]


test70_df = pd.read_pickle('data/tx_depistage70.pkl')

kpi_df = kpi_df.merge(test_df, how='left')\
    .merge(test70_df, how='left')\
    .rename(columns={'rolling_test_100k':'tx_depistage'})

kpi_df.tail()

,reg,libelle_reg,libelle_dep,jour,dom_tom,incid_tous,incid_70+,rea%,rea%_dep,niveau_global,tx_depistage,tx_despitage_moins69,tx_depistage_70+
26356,11,Île-de-France,Yvelines,2020-11-29,False,77.03,129.91,69.75,72.62,Alerte,742.54,716.83,981.62
26357,11,Île-de-France,Yvelines,2020-11-30,False,69.85,113.25,68.70,71.43,Alerte,715.88,692.55,935.54
26358,11,Île-de-France,Yvelines,2020-12-01,NaN,NaN,NaN,66.78,70.24,NaN,NaN,NaN,NaN
26359,11,Île-de-France,Yvelines,2020-12-02,NaN,NaN,NaN,65.21,69.05,NaN,NaN,NaN,NaN
26360,11,Île-de-France,Yvelines,2020-12-03,NaN,NaN,NaN,62.34,66.67,NaN,NaN,NaN,NaN


# Set variables

In [3]:
# Data sources
# TODO: make into dict with metrics as keys

source_incid = kpi.source
source_rea = hd.source
source_proc = 'https://github.com/limegimlet/covid19'

print("Sources for raw data & processing")
for source in [source_incid, source_rea, source_proc]:
    print(source)

# set confinement date, for calculating % change
confined = '2020-10-28'

# for more user-friendly plot titles
metric_dict = {'incid_tous': "Taux d'incidence",
               'incid_70+': "Taux d'incidence des personnes âgées",
               'rea%_dep': "Taux d'occupation en réa",
                'tx_depistage': "Taux de dépistage"}


Sources for raw data & processing
https://www.data.gouv.fr/fr/datasets/donnees-relatives-aux-resultats-des-tests-virologiques-covid-19/
https://www.data.gouv.fr/en/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/
https://github.com/limegimlet/covid19


# Helper functions

In [24]:
def df_to_md_table(df):
    '''converts df to markdown. 
    Used for pasting into an .MD file'''
    
    from IPython.display import Markdown, display
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    display(Markdown(df_formatted.to_clipboard(sep="|", index=True)))


def create_projection_line(metric, n, start_x, start_y, end_x, end_y, dash):
    
    trace_proj = go.Scatter(x=[start_x, end_x], 
                            y=[start_y, end_y],  
                            mode='lines', 
                            line_color='darkgray',
                            name='{}-day chg rate'.format(n),
                            #hovertext=end_x,
                            hovertemplate="Est. date: " + end_x,
                            line_dash=dash,
                            line_width=1)
    
    return trace_proj

def project_target(metric, target_val, df, target_date='2020-12-15', date_fmt='%Y-%m-%d'):
    latest_date = df.index.max()
    latest_val = df.loc[latest_date][metric]

    # Required change rate to meet goal
    chg_reqd = int(latest_val - target_val)
    pct_chg_reqd = chg_reqd/latest_val
    days_left = (datetime.strptime(target_date, date_fmt) - datetime.strptime(latest_date, date_fmt)).days
    daily_pct_chg_reqd = round(pct_chg_reqd/days_left, 3)


    # Compare with actual change rate

    print("=== Projected days to reach {} {} ===\n".format(target_val, metric))
    print("To meet this, we need to decrease daily {} by {} between now ({}) and {}.".format(metric, chg_reqd, latest_date, target_date))
    print("This will require a minimum {} average daily change over next {} days.\n".format(daily_pct_chg_reqd, days_left))

    rows = []
    traces = []
    ndays = [14, 7, 5, 3]
    dashes = ['dot', 'dashdot', 'dash', 'solid']
    n_tups = list(zip(ndays, dashes))
    
    for tup in n_tups:
        n = tup[0]
        dash = tup[1]
        n_days_prev = df.index[-n]
        n_days_prev_val = df.loc[n_days_prev][metric]
        chg_actual = n_days_prev_val - latest_val
        pct_chg_actual = chg_actual/n_days_prev_val
        daily_chg_actual = round(pct_chg_actual/n, 3)

        # how long to reach target?
        est_days_target = int(round(pct_chg_reqd/daily_chg_actual,0))
        est_target_date = datetime.strftime((datetime.strptime(latest_date, date_fmt) + timedelta(est_days_target)), date_fmt)

        #print("Using {}-day chg rate of {}: {} days, or {}\n".format(n, daily_chg_actual, est_days_target, est_target_date))
        n_row = ['{} days'.format(n), daily_chg_actual, est_days_target, est_target_date]
        rows.append(n_row)
        
        # create traces for each row
        trace = create_projection_line(metric, n, n_days_prev, n_days_prev_val, est_target_date, target_val, dash)
        traces.append(trace)

    labels = ['Estimate based on previous', 'Avg. daily change rate', 'Est. days to reach target', 'Est. target date']
    project_df = pd.DataFrame(data=rows, columns=labels)

    return project_df, traces

def plot_progress(plot_col, plot_df, target_trace, proj_traces):
    color_seq = [color_map[plot_col]]
    title = "{}: will it reach its target by Dec 15?".format(plot_col)
    x_min = plot_df['jour'].min()
    x_max = plot_df['jour'].max()
    fig = px.bar(plot_df, x = 'jour', y = plot_col, color_discrete_sequence=color_seq,
                 #facet_row = 'metric', facet_row_spacing=0.15, 
                 range_x=['2020-10-28', '2020-12-15'], title=title)
    
    fig.add_trace(target_trace) # target to reach by dec 15 
    fig.add_traces(proj_traces) # projections using different daily change rates
    fig.update_xaxes(tickangle=45)
    
    return fig

In [5]:
# export df as image
#fname = "conf_chg_summary.png"
#dfi.export(new_summary,"../covid_dataviz/img/{}".format(fname), )

# Plot progress to Dec 15

In [122]:
# create national pos df
df = pt.create_testing_df().groupby('jour')[['pos']].sum().rolling(7).mean().round()#.reset_index()
#df.iplot()

# get rea raw occ numbers
hosp_df = hd.get_hosp_data()

# create national rea df
rea_fr = hosp_df.groupby('jour').sum()[['rea']]#.reset_index()
#rea_fr.iplot()

# join the two
plot_df = df.join(rea_fr, how='outer').loc['2020-10-28':]
plot_df.columns = ["Positive cases, 7-day avg", 'Covid patients in ICU']
#plot_df = plot_df.stack().reset_index().rename(columns={"level_1":'metric', 0:'value'})
plot_df = plot_df.reset_index()

In [15]:
pal = py.colors.qualitative.Plotly
color_map = {"Positive cases, 7-day avg":pal[0], 
             'Covid patients in ICU':pal[1]}
x_min = plot_df['jour'].min()

# create target lines
pos_target = go.Scatter(x=[x_min, '2020-12-15'], 
                            y=[5000,5000],  
                            mode='lines', 
                            line_color='black',
                            name='Cases target',
                            line_dash = 'dash')

rea_target = go.Scatter(x=[x_min, '2020-12-15'], 
                            y=[3000,3000],  
                            mode='lines', 
                            line_color='black',
                            name='ICU target',
                            line_dash='dot')
                            #showlegend=False)

# Daily positive cases

In [16]:
#metric = 'pos'
#target_val = 3500
#from IPython.display import display

#for target_val in [4500, 4000, 3500, 3000, 2500, 2000]:

    #estimated_pos_dates, pos_traces = project_target(metric, target_val, df)
    #display(estimated_pos_dates)
    #print('\n')
#pos_traces

In [25]:
metric = 'pos'
target_val = 5000

estimated_pos_dates, pos_traces = project_target(metric, target_val, df)
display(estimated_pos_dates)
print('\n')
#pos_traces

=== Projected days to reach 5000 pos ===

To meet this, we need to decrease daily pos by 3862 between now (2020-11-30) and 2020-12-15.
This will require a minimum 0.029 average daily change over next 15 days.



,Estimate based on previous,Avg. daily change rate,Est. days to reach target,Est. target date
0,14 days,0.038,11,2020-12-11
1,7 days,0.032,14,2020-12-14
2,5 days,0.023,19,2020-12-19
3,3 days,0.015,29,2020-12-29


In [123]:
plot_col = "Positive cases, 7-day avg"
target_trace = pos_target
proj_traces = pos_traces

pos_fig = plot_progress(plot_col, plot_df, target_trace, proj_traces)
pos_fig.update_layout(title="<b>{}</b>: <br>Decrease has stopped. If trend continues, 5000 cases impossible by Dec 15".format(plot_col))
pos_fig.show()
fname = "{}_progress".format(metric)
#to_chart_studio(pos_fig, fname)

***Not shown above: this trend continues to at least Dec 3***

In other words, we still have the same number of daily positives to shed (~4000) within an ever-narrowing time frame.

## Fill in the gap from Dec 1 to 3

The challenge with reporting on the SPF data on data.gouv.fr is that they have a 3-day lag. That's why as of tge evening of Dec 3, the positve case numbers shown here only go up to Nov 30. 

However, I can manually get the raw daily numbers from the nighly updates on the Covid-19 page of the SPF website. Every night around 20h they update the site with new positive tests reported in the previous 24 hours. 

I've begun taking screenshots. Yes, screenshots, because this info gets overwritten every night. 

(And no, in answer to you codeaholics our there: I don't want to futz with creating a webscraper for 2 bullet points).

Incidentally, this is also the only way to get numbers for positive **antigen** tests, which are not shown in my plots, or anyone else's, since only PCR test results are available to the public as downloadable data. 

Anyways, here's are the numbers from the cutting-edge screenshots:

In [103]:
# manually create df of latest SPF updates

dates = ['2020-11-30', '2020-12-01', '2020-12-02', '2020-12-03']
day_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday']
pcr_pos = [2483, 6201, 12071, 10657]
antigen_pos = [1522, 1882, 1993, 2039]

spf_updates = pd.DataFrame({'jour': dates, 'day': day_week, 'pos_pcr': pcr_pos, 'pos_antigen': antigen_pos})
spf_updates

,jour,day,pos_pcr,pos_antigen
0,2020-11-30,Monday,2483,1522
1,2020-12-01,Tuesday,6201,1882
2,2020-12-02,Wednesday,12071,1993
3,2020-12-03,Thursday,10657,2039


Now, let's plot PCR positives fro Nov 30 onwards. 

Note the positive PCR tests are the "raw" daily numbers, not rolling 7-day averages. 

In [104]:
title = 'Daily raw counts of positive PCR tests since Mon Nov 30'
spf_updates.set_index('day')[['pos_pcr']].iplot(kind='bar', 
                                    barmode='stack', 
                                   title=title)

## Why the daily positives vary so widely

Probably the first thing you notice is how much these fluctuate. This is the weekend effect. 

It happens every week: the new positives recorded for Saturday and Sunday, which SPF will report 24 hours later on Sunday & Monday, respectively, will be always lower. 

Meanwhile, at the labs there is furious data-entry for test results that piled up over the weekend. As a result, SPF's nightly updates for Tuesday and Wednesday will also include catching up on the weekend backlog.

### The solution: use rolling 7-day averages

This is why typically Covid plots (including mine) focus on the rolling 7-day average for new cases: that is, take the sum the preceding 7 days' of test numbers, then divide by 7 to get a "smoothed" number that makes the overall trend much more clear.

<blockquote>This day-to-day 'noisiness' is important to keep in mind when reading, watching, or listening to media reports on Covid in your community. If a number for new cases seems like a big drop or jump, it's likely it's the raw number, which does NOT account for weekend under-reporting and then the Tuesday and Wednesday catching-up.</blockquote>{: .notice--info}

In French these are ususally indicated by _sur semaine glissante_.

## Calculate rolling averages of PCR positives for last 3 days

Since we already know the rollling average for Nov 30 (8862) as well as the raw PCR positive counts for Dec 1 to 3, we can now calculate the 7-day rolling averages. 

`current_7d_avg = (previous_7d_avg * 6) + current_raw_ / 7`

I won't bother doing this for antigen tests as it's too early to tell if they suffer from the weekend effect as well. 

In [109]:
def fill_missing_rolling_avg(spf_updates):
    '''Calculates missing rolling averages using last known rolling average
    and subsequent days' raw counts'''
    
    empty_rows = spf_updates.index[1:].values
    for row in empty_rows:
        prev_roll_pcr_pos = spf_updates.loc[row-1]['rolling_pcr_pos']
        spf_updates.iloc[row, 4] = int(((prev_roll_pcr_pos * 6) + spf_updates.loc[row]['pos_pcr'])/7)
    
    return spf_updates

In [108]:
# calculate the rolling 7-day avg

latest_rolling_pos_date = '2020-11-30'
latest_rolling_pos_val = plot_df.loc[plot_df['jour']==latest_rolling_pos_date][plot_col].values[0]

spf_updates['rolling_pcr_pos'] = [latest_rolling_pos_val, np.nan, np.nan, np.nan]

spf_updates = fill_missing_rolling_avg(spf_updates)
spf_updates

,jour,day,pos_pcr,pos_antigen,rolling_pcr_pos
0,2020-11-30,Monday,2483,1522,8862.0
1,2020-12-01,Tuesday,6201,1882,8481.0
2,2020-12-02,Wednesday,12071,1993,8993.0
3,2020-12-03,Thursday,10657,2039,9230.0


To keep things clean, let's now drop the raw PCR positives:

In [115]:
#spf_updates = spf_updates.drop('pos_pcr', axis=1)
spf_updates

,jour,day,pos_antigen,rolling_pcr_pos
0,2020-11-30,Monday,1522,8862.0
1,2020-12-01,Tuesday,1882,8481.0
2,2020-12-02,Wednesday,1993,8993.0
3,2020-12-03,Thursday,2039,9230.0


OK time to plot again, this time the rolling PCR positives.

In [117]:
title = "Once you look at the <b>rolling 7-day average</b>, daily PCR postives are stagnating"
plot_df = spf_updates.set_index('day')[['rolling_pcr_pos']]

plot_df.iplot(kind='bar',
              title=title)

Rolling PCR postives for Thursday Dec 3 was 9230, so it's back to the same value as Nov 29. 

And what happens if we can antigen test results to the pcr tests?

In [118]:
title = "Total of 7d-rolling daily PCR postives +  daily antigen positives"
plot_df = spf_updates.set_index('day')[['rolling_pcr_pos', 'pos_antigen']]

plot_df.iplot(kind='bar',
              barmode='stack',
              title=title)

Now you can see a clear upward trend

# Current ICU occupancy 

In [19]:
metric = 'rea'
target_val = 3000

estimated_rea_dates, rea_traces = project_target(metric, target_val, rea_fr)
estimated_rea_dates

=== Projected days to reach 3000 rea ===

To meet this, we need to decrease daily rea by 395 between now (2020-12-03) and 2020-12-15.
This will require a minimum 0.01 avg daily chg over next 12 days.



,Estimate based on previous,Avg. daily change rate,Est. days to reach target,Est. target date
0,14 days,0.018,6,2020-12-09
1,7 days,0.017,7,2020-12-10
2,5 days,0.018,6,2020-12-09
3,3 days,0.017,7,2020-12-10


In [20]:
plot_col = 'Covid patients in ICU'
target_trace = rea_target
proj_traces = rea_traces

rea_fig = plot_progress(plot_col, plot_df, target_trace, proj_traces)
rea_fig.update_layout(title="<b>{}</b>: <br>Currently on track for Dec 15 target".format(plot_col))
rea_fig.show()

fname = "{}_progress".format(metric)
#to_chart_studio(rea_fig, fname)